## Imports and constants

to do: 

transformers, features generation,
fix left bisect as described in the changed rules

In [2]:
SPLIT_SEED = 42
KFOLD_SPLITS = 2

IS_EMBEDDING_USED = False
if (IS_EMBEDDING_USED):
    IS_GPU_USED = False
    FAISS_ITERATIONS = 2
    FACTORS = 10
    
IS_POLARS_USED = False
    
IS_NOTEBOOK_ON_KAGGLE = False
IS_NOTBOOK_ON_COLAB = True

IS_MODEL_TRANSFORMERS4REC = True
IS_MODEL_CATBOOST = False
IS_MODEL_RANDFOREST = False

In [3]:
!pip install feather-format >> none -q
!pip install polars -q

import bisect
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.express as px
import polars as pl
import pyarrow as pa
import pyarrow.parquet as pq
import seaborn as sns
import sklearn.metrics as m
import sys
import tensorflow as tf
import time
import warnings

from pathlib import Path
from sklearn.model_selection import KFold

if IS_EMBEDDING_USED:
    !pip install faiss-cpu --no-cache -q
    
    import implicit
    import scipy

if IS_MODEL_RANDFOREST:
    from sklearn.ensemble import RandomForestClassifier

if IS_MODEL_CATBOOST:
    from catboost import CatBoostClassifier, CatBoostRegressor, Pool

if (IS_MODEL_TRANSFORMERS4REC):
    !pip install transformers4rec -q
    !pip install torch -q
    !pip install nvtabular -q
    !pip install torchmetrics -q
    !pip install merlin-dataloader -q
    
    os.environ["CUDA_VISIBLE_DEVICES"]="0"

    import glob
    import merlin
    import torch 

    from merlin_standard_lib import Schema
    from merlin.io import Dataset
    from transformers4rec import torch as tr
    from transformers4rec.torch.ranking_metric import NDCGAt, AvgPrecisionAt, RecallAt
    from transformers4rec.torch.utils.examples_utils import wipe_memory

%matplotlib inline
sns.set_style('darkgrid')
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 61.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 KB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 55.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.9/dist-packages/merlin/dtypes/mappings/cudf.py:57: UserWarning: cuDF dtype mappings did not load successfully due to an error: No module named 'cudf'
  warn(f"cuDF dtype mappings did not load successfully due to an error: {exc.msg}")
/usr/local/lib/python3.9/dist-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


In [8]:
if IS_NOTEBOOK_ON_KAGGLE:
    LOCAL_DATA_PATH = Path('/kaggle', 'input', 'mts-ml-cookies')
    DATA_FILE = Path('dataset_full.feather')
    TARGET_FILE = Path('target_train.feather')
    SUBMISSION_FILE = Path('submission.feather')
elif IS_NOTBOOK_ON_COLAB:
    LOCAL_DATA_PATH = Path('/content', 'mts-ml-cookies')
    DATA_FILE = Path('dataset_full.feather')
    TARGET_FILE = Path('target_train.feather')
    SUBMISSION_FILE = Path('submission.feather')

## Showing data

In [9]:
id_to_submit = pd.read_feather(Path(LOCAL_DATA_PATH, SUBMISSION_FILE))
display(id_to_submit.head(3))
id_to_submit.info

,user_id
0,221301
1,31271
2,211594


<bound method DataFrame.info of         user_id
0        221301
1         31271
2        211594
3        253119
4        192578
...         ...
144719   415284
144720   415285
144721   415286
144722   415306
144723   415315

[144724 rows x 1 columns]>

In [11]:
data = pd.read_feather(Path(LOCAL_DATA_PATH, DATA_FILE))
display(data.head())
data.describe()

ArrowInvalid: ignored

In [ ]:
if IS_POLARS_USED:
    for column in data.columns:
        if column != "date" and column !="price" and column !="user_id" and column !="request_cnt":
            print(type(data[column]))
            data[column] = data[column].cat.codes
        elif column == "date":
            data["date"] = pd.to_datetime(data["date"])

    data.head()

In [ ]:
if IS_POLARS_USED:
    data = pl.from_pandas(data)
    display(data.select(['cpe_type_cd']).to_pandas()['cpe_type_cd'].value_counts())
    data['cpe_type_cd'].value_counts()

In [ ]:
targets = pd.read_feather(Path(LOCAL_DATA_PATH, TARGET_FILE))
display(targets.head())
targets = pa.Table.from_pandas(targets)
pd.DataFrame([(z.name, z.type) for z in targets.schema], columns = [['field', 'type']])

## Counting users <-> items embedding

In [ ]:
%%time
if IS_EMBEDDING_USED:
    data_agg = data.groupby(['user_id', 'url_host'], maintain_order=True).agg(
        pl.sum("request_cnt")
    )
    url_set = set(data_agg.select(['url_host']).to_pandas()['url_host'])
    print(f'{len(url_set)} urls')
    url_dict = {url: idurl for url, idurl in zip(url_set, range(len(url_set)))}
    usr_set = set(data_agg.select(['user_id']).to_pandas()['user_id'])
    print(f'{len(usr_set)} users')
    usr_dict = {usr: user_id for usr, user_id in zip(usr_set, range(len(usr_set)))}

In [ ]:
%%time
if IS_EMBEDDING_USED:
    values = np.array(data_agg['request_cnt'].to_pandas())
    rows = np.array(data_agg['user_id'].to_pandas().map(usr_dict))
    cols = np.array(data_agg['url_host'].to_pandas().map(url_dict))
    mat = scipy.sparse.coo_matrix((values, (rows, cols)), shape=(rows.max() + 1, cols.max() + 1))
    als = implicit.approximate_als.FaissAlternatingLeastSquares(
        factors = FACTORS, iterations = FAISS_ITERATIONS, use_gpu = IS_GPU_USED, calculate_training_loss = True, regularization = 0.1
    )

In [ ]:
%%time
if IS_EMBEDDING_USED:
    als.fit(mat)

In [ ]:
if IS_EMBEDDING_USED:
    u_factors = als.user_factors 
    print(len(u_factors))

In [ ]:
if IS_EMBEDDING_USED:
    d_factors = als.item_factors
    print(len(d_factors))

## Data loading with merlin

In [ ]:
train = Dataset(data)
schema = train.schema
schema

In [ ]:
inputs = tr.TabularSequenceFeatures.from_schema(
        schema,
        max_sequence_length=20,
        continuous_projection=64,
        masking="mlm",
        d_output=100,
)

In [ ]:
# Define XLNetConfig class and set default parameters for HF XLNet config  
transformer_config = tr.XLNetConfig.build(
    d_model=64, n_head=4, n_layer=2, total_seq_length=20
)
# Define the model block including: inputs, masking, projection and transformer block.
body = tr.SequentialBlock(
    inputs, tr.MLPBlock([64]), tr.TransformerBlock(transformer_config, masking=inputs.masking)
)

# Define the evaluation top-N metrics and the cut-offs
metrics = [NDCGAt(top_ks=[20, 40], labels_onehot=True),  
           RecallAt(top_ks=[20, 40], labels_onehot=True)]

# Define a head related to next item prediction task 
head = tr.Head(
    body,
    tr.NextItemPredictionTask(weight_tying=True, 
                              metrics=metrics),
    inputs=inputs,
)

# Get the end-to-end Model class 
model = tr.Model(head)

In [ ]:
per_device_train_batch_size = int(os.environ.get(
    "per_device_train_batch_size", 
    '128'
))

per_device_eval_batch_size = int(os.environ.get(
    "per_device_eval_batch_size", 
    '32'
))

In [ ]:
from transformers4rec.config.trainer import T4RecTrainingArguments
from transformers4rec.torch import Trainer
# Set hyperparameters for training 
train_args = T4RecTrainingArguments(data_loader_engine='merlin', 
                                    dataloader_drop_last = True,
                                    gradient_accumulation_steps = 1,
                                    per_device_train_batch_size = per_device_train_batch_size, 
                                    per_device_eval_batch_size = per_device_eval_batch_size,
                                    output_dir = "./tmp", 
                                    learning_rate=0.0005,
                                    lr_scheduler_type='cosine', 
                                    learning_rate_num_cosine_cycles_by_epoch=1.5,
                                    num_train_epochs=5,
                                    max_sequence_length=20, 
                                    report_to = [],
                                    logging_steps=50,
                                    no_cuda=False)

In [ ]:
# Instantiate the T4Rec Trainer, which manages training and evaluation for the PyTorch API
trainer = Trainer(
    model=model,
    args=train_args,
    schema=schema,
    compute_metrics=True,
)

In [ ]:
start_window_index = int(os.environ.get(
    "start_window_index", 
    '1'
))

final_window_index = int(os.environ.get(
    "final_window_index", 
    '8'
))

In [ ]:
start_time_window_index = start_window_index
final_time_window_index = final_window_index
#Iterating over days of one week
for time_index in range(start_time_window_index, final_time_window_index):
    # Set data 
    time_index_train = time_index
    time_index_eval = time_index + 1
    train_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_train}/train.parquet"))
    eval_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_eval}/valid.parquet"))
    print(train_paths)
    
    # Train on day related to time_index 
    print('*'*20)
    print("Launch training for day %s are:" %time_index)
    print('*'*20 + '\n')
    trainer.train_dataset_or_path = train_paths
    trainer.reset_lr_scheduler()
    trainer.train()
    trainer.state.global_step +=1
    print('finished')
    
    # Evaluate on the following day
    trainer.eval_dataset_or_path = eval_paths
    train_metrics = trainer.evaluate(metric_key_prefix='eval')
    print('*'*20)
    print("Eval results for day %s are:\t" %time_index_eval)
    print('\n' + '*'*20 + '\n')
    for key in sorted(train_metrics.keys()):
        print(" %s = %s" % (key, str(train_metrics[key]))) 
    wipe_memory()

In [ ]:
eval_data_paths = glob.glob(os.path.join(OUTPUT_DIR, f"{time_index_eval}/valid.parquet"))

In [ ]:
# set new data from day 7
eval_metrics = trainer.evaluate(eval_dataset=eval_data_paths, metric_key_prefix='eval')
for key in sorted(eval_metrics.keys()):
    print("  %s = %s" % (key, str(eval_metrics[key])))

In [ ]:
model_path= os.environ.get("OUTPUT_DIR", f"{INPUT_DATA_DIR}/saved_model")
model.save(model_path)

## Sex prediction

In [ ]:
id_to_submit = pd.read_feather(f'{LOCAL_DATA_PATH}/{SUBMISSION_FILE}')

In [ ]:
if IS_EMBEDDING_USED:
    inv_usr_map = {v: k for k, v in usr_dict.items()}
    usr_emb = pd.DataFrame(d_factors)
    usr_emb['user_id'] = usr_emb.index.map(inv_usr_map)
    usr_targets = targets.to_pandas()
    df = usr_targets.merge(usr_emb, how = 'inner', on = ['user_id'])
    df = df[df['is_male'] != 'NA']
    df = df.dropna()
    df['is_male'] = df['is_male'].map(int)
    print(df['is_male'].value_counts())

#### Model trainings

In [ ]:
max_gini = 0
train_index = 0
test_index = 0

if IS_MODEL_CATBOOST:
    clf = CatBoostClassifier()
elif IS_MODEL_RANDFOREST:
    clf = RandomForestClassifier(random_state=SPLIT_SEED)

In [ ]:
%%time
if IS_MODEL_CATBOOST or IS_MODEL_RANDFOREST:
    x = df.drop(['user_id', 'age', 'is_male'], axis = 1)
    y = df['is_male']

    kf = KFold(n_splits=KFOLD_SPLITS)

    for train_index, test_index in kf.split(x):
        print("Train indexes: ", train_index[:5],
              "; Test indexes: ", test_index[:5], ";\n",
              "Train length: ", len(train_index), 
              "; Test length: ", len(test_index), 
              "; Dataset length: ", len(x), ';', sep='', end='\n')
        
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if IS_MODEL_CATBOOST:
            clf.fit(x_train, y_train, verbose = False)
        elif IS_MODEL_RANDFOREST:
            clf.fit(x_train, y_train)
        
        gini = float(f'{2 * m.roc_auc_score(y_test, clf.predict_proba(x_test)[:,1]) - 1:2.3f}')
        if gini > max_gini:
            max_gini = gini
            max_train_index = train_index
            max_test_index = test_index
        
        print(f'Current GINI is {gini}, current max is {max_gini}', end="\n\n")

In [ ]:
print(f'Max GINI is {max_gini}')
print(f'Max train index is {max_train_index[:5]}')
print(f'Max test index is {max_test_index[:5]}')

have to use predict_proba because we need probability of the sex

In [ ]:
if IS_MODEL_CATBOOST or IS_MODEL_RANDFOREST:
    if IS_MODEL_CATBOOST:
        clf.fit(x.iloc[max_train_index], y.iloc[max_train_index], verbose = False)
    elif IS_MODEL_RANDFOREST:
        clf.fit(x.iloc[max_train_index], y.iloc[max_train_index])
    id_to_submit['user_id'].unique
    id_to_submit['is_male'] = clf.predict_proba(id_to_submit.merge(usr_emb, how = 'left', on = ['user_id']).drop(['user_id'], axis = 1))[:,1]
    
id_to_submit.head()

In [ ]:
print("Current GINI score: ", float(f'{2 * m.roc_auc_score(y.iloc[max_test_index], clf.predict_proba(x.iloc[max_test_index])[:,1]) - 1:2.3f}'))

## Age prediction

In [ ]:
def age_bucket(x):
    return bisect.bisect_left([18,25,35,45,55,65], x)

In [ ]:
df = usr_targets.merge(usr_emb, how = 'inner', on = ['user_id'])
df = df[df['age'] != 'NA']
df = df.dropna()
df['age'] = df['age'].map(age_bucket)
sns.histplot(df['age'], bins = 7)

In [ ]:
max_f1 = 0
train_index = 0
test_index = 0

if IS_MODEL_CATBOOST:
    clf = CatBoostClassifier()
elif IS_MODEL_RANDFOREST:
    clf = RandomForestClassifier(random_state=SPLIT_SEED)

In [ ]:
if IS_MODEL_CATBOOST or IS_MODEL_RANDFOREST:
    x = df.drop(['user_id', 'age', 'is_male'], axis = 1)
    y = df['age']

    kf = KFold(n_splits=KFOLD_SPLITS)

    for train_index, test_index in kf.split(x):
        print("Train indexes: ", train_index[:5],
              "; Test indexes: ", test_index[:5], ";\n",
              "Train length: ", len(train_index), 
              "; Test length: ", len(test_index), 
              "; Dataset length: ", len(x), ';', sep='', end='\n')
        
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        if IS_MODEL_CATBOOST:
            clf.fit(x_train, y_train, verbose = False)
        elif IS_MODEL_RANDFOREST:
            clf.fit(x_train, y_train)
        
        f1 = m.f1_score(y_test, clf.predict(x_test), average='weighted')
        if f1 > max_f1:
            max_f1 = f1
            max_train_index = train_index
            max_test_index = test_index
        
        print(f'Current F1 is {f1}, current max is {max_f1}', end="\n\n")

In [ ]:
print(f'Max F1 is {max_f1}')
print(f'Max train index is {max_train_index[:5]}')
print(f'Max test index is {max_test_index[:5]}')

In [ ]:
if IS_MODEL_CATBOOST or IS_MODEL_RANDFOREST:
    if IS_MODEL_CATBOOST:
        clf.fit(x.iloc[max_train_index], y.iloc[max_train_index], verbose = False)
    elif IS_MODEL_RANDFOREST:
        clf.fit(x.iloc[max_train_index], y.iloc[max_train_index])
    id_to_submit['age'] = clf.predict(id_to_submit[['user_id']].merge(usr_emb, how = 'left', on = ['user_id']).drop(['user_id'], axis = 1))
    
id_to_submit.head()

In [ ]:
if IS_MODEL_CATBOOST or IS_MODEL_RANDFOREST:
    print(m.classification_report(
          y.iloc[max_test_index], 
          clf.predict(x.iloc[max_test_index]), 
          target_names = ['<18', '18-25','25-34', '35-44', '45-54', '55-65', '65+']
    ))

## Submission

In [ ]:
display(id_to_submit.head())
display(id_to_submit.is_male.value_counts())
display(id_to_submit.age.value_counts())
id_to_submit.to_csv('submission.csv', index = False)

In [ ]:
print(f'Score: {max_gini + 2*max_f1}')